In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F    # relu, tanh 

from torch.utils.data import DataLoader    # easier dataset management 
import torchvision.datasets as datasets
import torchvision.transforms as transforms

### create fully connected network

In [24]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):    # (28 * 28 = 784)
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# testing
model = NN(784, 10)
x = torch.randn(64, 784)   # 64 == number of images, 784 == number of pixels
print(model(x).shape)

In [30]:
class CNN(nn.Module):
    def __init__(self, in_channel=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))    # same convolution
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))  
        self.fc1 = nn.Linear(16*7*7, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)    # number of minibatch we send in, -1
        x = self.fc1(x)
        return x

# set device 
model = CNN()
x = torch.randn(64, 1, 28, 28)
print(model(x).shape)


torch.Size([64, 10])


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### hyperparameters

In [37]:
in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### load data

In [38]:
train_dataset = datasets.MNIST(root = 'dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root = 'dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [39]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # get data to cuda if possible 
        data = data.to(device)
        targets = targets.to(device)

        # forward 
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()    # set all gradients to zero for each batch
        loss.backward()

        # gradient descent or adap step 
        optimizer.step()


NameError: name 'criterion' is not defined